In [5]:
import torch
import torch.nn as nn
from torch.nn import functional as F

device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)

batch_size = 32
block_size = 128
max_iters = 200
learning_rate = 2e-5
eval_iters = 100
n_embd = 384
n_head = 4
n_layer = 4
dropout = 0.2

cpu


embeddig vector for sad and happy <br>
sad = [0.2,0.8]
happy = [0.9,0.8]

0.8 shows emotions
0.2/0.9 shows intensity

In [6]:
with open('wizard_of_oz.txt','r',encoding='utf-8') as f:
    text = f.read()

print(len(text))


232315


In [7]:
chars = sorted(set(text))
print(chars)
vocab_size = len(chars)

['\n', ' ', '!', '"', '&', "'", '(', ')', '*', ',', '-', '.', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '[', ']', '_', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '\ufeff']


Tokenization

In [8]:
string_to_int = {ch:i for i,ch in enumerate(chars)} 
int_to_string = {i:ch for i,ch in enumerate(chars)}
encode = lambda s: [string_to_int[ch] for ch in s]
decode = lambda x: ''.join([int_to_string[i] for i in x])       

In [9]:
print(encode('hello'))  

[61, 58, 65, 65, 68]


In [10]:
decode(encode('hello')) 

'hello'

In [11]:
data = torch.tensor(encode(text),dtype=torch.long)  
data[:100]

tensor([80,  0,  0, 28, 39, 42, 39, 44, 32, 49,  1, 25, 38, 28,  1, 44, 32, 29,
         1, 47, 33, 50, 25, 42, 28,  1, 33, 38,  1, 39, 50,  0,  0,  1,  1, 26,
        49,  0,  0,  1,  1, 36, 11,  1, 30, 42, 25, 38, 35,  1, 26, 25, 45, 37,
         0,  0,  1,  1, 25, 45, 44, 32, 39, 42,  1, 39, 30,  1, 44, 32, 29,  1,
        47, 33, 50, 25, 42, 28,  1, 39, 30,  1, 39, 50,  9,  1, 44, 32, 29,  1,
        36, 25, 38, 28,  1, 39, 30,  1, 39, 50])

train-val split

In [12]:
n = 0.8
train_data = data[:int(n*len(data))]    
val_data = data[int(n*len(data)):] 

def get_batch(split):
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data)-block_size,(batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix]) 
    x, y = x.to(device), y.to(device)
    return x,y

x,y = get_batch('train')   
print("Input",x)
print("Target",y)

Input tensor([[ 1, 57, 58,  ...,  1, 73, 61],
        [72,  9,  3,  ..., 73,  1,  5],
        [69, 65, 58,  ..., 69, 58, 68],
        ...,
        [ 1, 57, 68,  ..., 67,  1, 75],
        [65, 65,  1,  ..., 74, 60, 61],
        [68, 65, 64,  ..., 61, 68, 65]])
Target tensor([[57, 58, 75,  ..., 73, 61, 58],
        [ 9,  3,  1,  ...,  1,  5, 31],
        [65, 58,  1,  ..., 58, 68, 69],
        ...,
        [57, 68,  1,  ...,  1, 75, 62],
        [65,  1, 72,  ..., 60, 61, 73],
        [65, 64, 72,  ..., 68, 65, 58]])


In [ ]:
# block_size = 8
x = train_data[:block_size]
y = train_data[1:block_size+1]  

for t in range(block_size):
    context = x[:t+1]
    target = y[t]   

tensor([80]) tensor(0)
tensor([80,  0]) tensor(0)
tensor([80,  0,  0]) tensor(28)
tensor([80,  0,  0, 28]) tensor(39)
tensor([80,  0,  0, 28, 39]) tensor(42)
tensor([80,  0,  0, 28, 39, 42]) tensor(39)
tensor([80,  0,  0, 28, 39, 42, 39]) tensor(44)
tensor([80,  0,  0, 28, 39, 42, 39, 44]) tensor(32)
tensor([80,  0,  0, 28, 39, 42, 39, 44, 32]) tensor(49)
tensor([80,  0,  0, 28, 39, 42, 39, 44, 32, 49]) tensor(1)
tensor([80,  0,  0, 28, 39, 42, 39, 44, 32, 49,  1]) tensor(25)
tensor([80,  0,  0, 28, 39, 42, 39, 44, 32, 49,  1, 25]) tensor(38)
tensor([80,  0,  0, 28, 39, 42, 39, 44, 32, 49,  1, 25, 38]) tensor(28)
tensor([80,  0,  0, 28, 39, 42, 39, 44, 32, 49,  1, 25, 38, 28]) tensor(1)
tensor([80,  0,  0, 28, 39, 42, 39, 44, 32, 49,  1, 25, 38, 28,  1]) tensor(44)
tensor([80,  0,  0, 28, 39, 42, 39, 44, 32, 49,  1, 25, 38, 28,  1, 44]) tensor(32)
tensor([80,  0,  0, 28, 39, 42, 39, 44, 32, 49,  1, 25, 38, 28,  1, 44, 32]) tensor(29)
tensor([80,  0,  0, 28, 39, 42, 39, 44, 32, 49,  1, 

In [31]:
@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

In [32]:
class Head(nn.Module):
    """ one head of self-attention """

    def __init__(self, head_size):
        super().__init__()
        self.key = nn.Linear(n_embd, head_size, bias=False)
        self.query = nn.Linear(n_embd, head_size, bias=False)
        self.value = nn.Linear(n_embd, head_size, bias=False)
        self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))

        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        # input of size (batch, time-step, channels)
        # output of size (batch, time-step, head size)
        B,T,C = x.shape
        k = self.key(x)   # (B,T,hs)
        q = self.query(x) # (B,T,hs)
        # compute attention scores ("affinities")
        wei = q @ k.transpose(-2,-1) * k.shape[-1]**-0.5 # (B, T, hs) @ (B, hs, T) -> (B, T, T)
        wei = wei.masked_fill(self.tril[:T, :T] == 0, float('-inf')) # (B, T, T)
        wei = F.softmax(wei, dim=-1) # (B, T, T)
        wei = self.dropout(wei)
        # perform the weighted aggregation of the values
        v = self.value(x) # (B,T,hs)
        out = wei @ v # (B, T, T) @ (B, T, hs) -> (B, T, hs)
        return out

# [1, 0, 0]
# [1, 0.6, 0]
# [1, 0.6, 0.4]

class MultiHeadAttention(nn.Module):
    """ multiple heads of self-attention in parallel """

    def __init__(self, num_heads, head_size):
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
        self.proj = nn.Linear(head_size * num_heads, n_embd)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        out = torch.cat([h(x) for h in self.heads], dim=-1) # (B, T, F) -> (B, T, [h1, h1, h1, h1, h2, h2, h2, h2, h3, h3, h3, h3])
        out = self.dropout(self.proj(out))
        return out

class FeedFoward(nn.Module):
    """ a simple linear layer followed by a non-linearity """

    def __init__(self, n_embd):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embd, 4 * n_embd),
            nn.ReLU(),
            nn.Linear(4 * n_embd, n_embd),
            nn.Dropout(dropout),
        )

    def forward(self, x):
        return self.net(x)

class Block(nn.Module):
    """ Transformer block: communication followed by computation """

    def __init__(self, n_embd, n_head):
        # n_embd: embedding dimension, n_head: the number of heads we'd like
        super().__init__()
        head_size = n_embd // n_head
        self.sa = MultiHeadAttention(n_head, head_size)
        self.ffwd = FeedFoward(n_embd)
        self.ln1 = nn.LayerNorm(n_embd)
        self.ln2 = nn.LayerNorm(n_embd)

    def forward(self, x):
        y = self.sa(x)
        x = self.ln1(x + y)
        y = self.ffwd(x)
        x = self.ln2(x + y)
        return x


class GPTLanguageModel(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size,n_embd)
        self.position_embedding_table = nn.Embedding(block_size,n_embd)
        self.blocks = nn.Sequential(*[Block(n_embd,n_head=n_head) for _ in range(n_layer)])
        self.ln_f = nn.LayerNorm(n_embd)
        self.lm_head = nn.Linear(n_embd,vocab_size) 

        self.apply(self._init_weights)

    def _init_weights(self, module):
        if isinstance(module,nn.Linear):
            torch.nn.init.normal_(module.weight, mean=0.0,std=0.02)
            if module.bias is not None:
                torch.nn.init.zeros_(module.bias)
        elif isinstance(module,nn.Embedding):
            torch.nn.init.normal_(module.weight,mean=0.0, std=0.02)
            

        
    def forward(self, index, targets=None):
        B, T= index.shape
        # idx and targets are both (B,T) tensor of integers
        tok_emb = self.token_embedding_table(index) # (B,T,C)
        pos_emb = self.position_embedding_table(torch.arange(T, device=device)) # (T,C)
        x = tok_emb + pos_emb # (B,T,C)
        x = self.blocks(x) # (B,T,C)
        x = self.ln_f(x) # (B,T,C)
        logits = self.lm_head(x) # (B,T,vocab_size)
        
        if targets is None:
            loss = None
        else:
            B, T, C= logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)
        
        return logits, loss
    
    def generate(self, index, max_new_tokens):
        # index is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # get the predictions
            logits, loss = self.forward(index)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            index_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            index = torch.cat((index, index_next), dim=1) # (B, T+1)
        return index


model = GPTLanguageModel(vocab_size)
m = model.to(device)

context = torch.zeros((1,1), dtype=torch.long, device=device)
generated_chars = decode(m.generate(context, max_new_tokens=50)[0].tolist())
print(generated_chars)


AekS,Q2pls0t!OQF!9
iI18q0 *(bk4rYu
AxhBgbKHQ73f!]h


In [33]:
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

for iter in range(max_iters):
    if iter % eval_iters == 0:
        losses = estimate_loss()
        print(f"step: {iter}, train loss: {losses['train']:.3f}, val loss: {losses['val']:.3f}")

    # sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = model.forward(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()
print(loss.item())

step: 0, train loss: 4.441, val loss: 4.439
step: 100, train loss: 2.917, val loss: 2.944
2.694988489151001


In [39]:
context = torch.zeros((1,1), dtype=torch.long, device=device)
generated_chars = decode(m.generate(context, max_new_tokens=100)[0].tolist())
print(generated_chars)


gDan thathenmp hyotheu3 utp alougrerend i" ur﻿ender[o!k Cd s, wottorDool '5otcofugethere! 4Y t *8d w
